In [15]:
import openai  # for making OpenAI API requests
from datasets import load_dataset
import tiktoken
import numpy as np
import re

openai.api_key = 'APY KEY'

In [16]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Encoding name	OpenAI models

# cl100k_base:	gpt-4, gpt-3.5-turbo, text-embedding-ada-002
# p50k_base:	text-davinci-003, text-davinci-002
# r50k_base:	GPT-3 models (text-curie-001, text-babbage-001, text-ada-001, davinci, curie, babbage, ada)

#print(num_tokens_from_string("Hello world, let's test tiktoken.", "cl100k_base"))

In [17]:
dataset = load_dataset("gsm8k", 'main')

In [18]:
thinking_styles = np.array([
    'Let’s think step by step.',  # Original
    'Let’s make a step by step plan and implement it with good notion and explanation',  # Original
    'How could I devise an experiment to help solve the problem?',  # Original
    'How can I simplify the problem so that it is easier to solve?',  # Original
    'Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.',  # Original
    'Critically evaluate the given information. Are there any biases or assumptions that might be influencing the interpretation?',  # Critical Thinking
    'Imagine a similar situation in a different time or place. How might it have been approached historically?',  # Historical Thinking
    'Project the outcomes into the future. If we follow this path, where might we end up in 10 years?',  # Futuristic Thinking
    'Put yourself in the shoes of all the stakeholders involved. How would they feel about this situation?',  # Empathetic Thinking
    'Consider how the different elements of this problem interact with one another. Can we identify any feedback loops or systemic issues?',  # Systems Thinking
    'Reflect on the ethical dimensions of this challenge. Are there moral principles that should guide our decision-making?',  # Ethical Thinking
    'Let’s compare this situation to a similar one in a different context. Are there lessons we can draw from that comparison?',  # Comparative Thinking
    'Consider the problem holistically. How do all the pieces fit together in the larger context?',  # Holistic Thinking
    'Trust your gut feeling on this. What does your intuition tell you about the best approach?',  # Intuitive Thinking
    'Examine the conflicting viewpoints around this issue. Can we find a middle ground or synthesis?',  # Dialectical Thinking
])

prompt_instruction = np.array([
    'Solve the math word problem, giving your answer as an arabic numeral',  
    'Solve the problem by explaining why systemic or structural issues would not be the cause of the issue.', 
    'Generate the answer to a word problem and write it as a number.',  
    'Break down the math problem step-by-step and provide the final numeric solution.', 
    'Apply fundamental mathematical principles to solve the problem, and provide a clear numeric answer.',
    'Use visual aids like diagrams or graphs if necessary to better illustrate the process of solving the problem.',
    'Consider using an algorithm or formula that is most efficient in solving this type of math problem. Provide the final answer as a number.',
    'Discuss the rationale behind each step you take in solving the problem, ensuring the explanation is clear and concise.',
    'If applicable, use real-world examples to contextualize the math problem, and solve it providing a numeric answer.',
    'Detail the mathematical concepts or theories utilized in solving the problem, and conclude with a numeric solution.',
    'Present multiple methods to solve the problem if possible, concluding each with a clear numeric answer.',
    'Ensure accuracy by double-checking calculations and logic used in solving the problem, and present the final numeric solution.',
    'Adapt a teaching approach, explaining the problem and solution as if educating someone with basic mathematical knowledge.',
])

prompt_mutators = np.array([
    'Generate a prompt mutant that introduces an element of suspense or intrigue.',  
    'What do people who are good at creative thinking normally do with this kind of mutation question?',  
    'Change the following instruction being as brief as possible',  
    'Modify the prompt to emphasize the importance of collaboration or teamwork in the process.',
    'Rework the instruction to appeal to someone with a background in arts or humanities.',
    'Twist the prompt to challenge someone with advanced expertise in the subject.',
    'Adjust the instruction to encourage out-of-the-box thinking or a novel approach.',
    'Refine the prompt to make it sound like a riddle or puzzle to be solved.',
    'Adapt the original instruction to cater to a younger audience or beginners.',
    'Convert the prompt to make it sound like a quest or mission.',
    'Reimagine the instruction as if it is a call to action for a global challenge or competition.',
    'Infuse humor or sarcasm into the original prompt.'
])


In [19]:

def create_prompt_examples(indexes, dataset):

    prompt_examples_str = 'These are some examples of similar problems and how to solve them: \n'

    for index in indexes:

        prompt_examples_str = prompt_examples_str + 'Q: ' + dataset['train'][int(index)]['question'] + '\nA: ' + dataset['train'][int(index)]['answer'] + '\n'

    return prompt_examples_str


def create_population(thinking_styles, instructions, prompt_examples, dataset, test_indexes):

    population =  []

    problems = dataset['test'][test_indexes]['question']

    answers = dataset['test'][test_indexes]['answer']

    for i in range(len(instructions)):

        prompt = thinking_styles[i] + '\n' + instructions[i] + '\n' + prompt_examples + '\nQ: ' + problems[i]

        population.append(prompt)


    return population, answers


def mutate_instructions(instructions, mutation_prompts, indexes_to_mutate, indexes_good_prompts):

    instructions_for_mutation, new_instructions = [], np.copy(instructions)

    if len(indexes_good_prompts) == 0:

        for i in range(len(indexes_to_mutate)):

            selected_instruction_index = indexes_to_mutate[np.random.randint(len(indexes_to_mutate))]

            instructions_for_mutation.append(mutation_prompts[np.random.randint(3)] + 
                                             '. The prompt you need to change is the following: \n' + 
                                             instructions[selected_instruction_index])


    elif len(indexes_good_prompts) > 0:

        for i in range(len(indexes_to_mutate)):

            selected_instruction_index = indexes_good_prompts[np.random.randint(len(indexes_good_prompts))]

            instructions_for_mutation.append(mutation_prompts[np.random.randint(3)] + ': \n' + instructions[selected_instruction_index])
    

    for i in range(len(instructions_for_mutation)):

        message = [{"role": "user", "content": instructions_for_mutation[i]}]

        response = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=message, temperature=0)

        new_instructions[indexes_to_mutate[i]] = response['choices'][0]['message']['content'] # type: ignore


    return new_instructions


def last_number(s):
    numbers = re.findall(r'\d+', s)
    return int(numbers[-1]) if numbers else None


def evaluate(prompts, answers):

    evaluations = np.zeros(len(answers))

    model_ans = ['']*len(answers)

    for i in range(len(prompts)):

        message = [{"role": "user", "content": prompts[i]}]

        response = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=message, temperature=0)

        model_ans[i] = response['choices'][0]['message']['content'] # type: ignore

        if last_number(answers[i]) == last_number(model_ans[i] ):

            evaluations[i] = 1
    
    return evaluations, model_ans


In [20]:
## INITIALIZATION

prompt_examples_indexes = np.random.randint(100, size=3)

initial_thinking_styles = thinking_styles[np.random.randint(15, size=5)]

initial_prompt_instruction = prompt_instruction[np.random.randint(12, size=5)]

initial_prompt_mutators = prompt_mutators[np.random.randint(11, size=5)]

prompt_examples = create_prompt_examples(prompt_examples_indexes, dataset)

test_problem_indexes = np.random.randint(100, size=5)

initial_population, initial_answers = create_population(initial_thinking_styles, initial_prompt_instruction, prompt_examples, dataset, test_problem_indexes)

num_generations = 7

In [21]:
print('Individual example:')
print()
print(initial_population[0])

Individual example:

Put yourself in the shoes of all the stakeholders involved. How would they feel about this situation?
Consider using an algorithm or formula that is most efficient in solving this type of math problem. Provide the final answer as a number.
These are some examples of similar problems and how to solve them: 
Q: James earns $20 an hour while working at his main job.  He earns 20% less while working his second job.  He works 30 hours at his main job and half that much at his second job.  How much does he earn per week?
A: James earns 20*.2=$<<20*.2=4>>4 less while working his second job
So he earns 20-4=$<<20-4=16>>16 an hour
At his first job he earns 20*30=$<<20*30=600>>600
He works 30/2=<<30/2=15>>15 hours at his second job
So he earns 15*16=$<<15*16=240>>240
So he earns 600+240=$<<600+240=840>>840 a week
#### 840
Q: Ann's favorite store was having a summer clearance. For $75 she bought 5 pairs of shorts for $7 each and 2 pairs of shoes for $10 each. She also bought 

In [22]:
# ALGORITHM
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

population = initial_population.copy()
answers = initial_answers.copy()
mutators = initial_prompt_mutators.copy()
thinking_styles = initial_thinking_styles.copy()
prompt_instructions = initial_prompt_instruction.copy()

for i in range(num_generations):
    
    evaluations, model_ans = evaluate(population, answers)

    good_indexes = np.where(evaluations==1)[0]
    bad_indexes = np.where(evaluations==0)[0]

    prompt_instructions = mutate_instructions(prompt_instructions, mutators, bad_indexes, good_indexes)

    population, answers = create_population(thinking_styles, prompt_instructions, prompt_examples, dataset, test_problem_indexes)

    population_fitness = np.mean(evaluations)

    print('Mean Generation', i, 'Fitness: ', population_fitness)
            

Mean Generation 0 Fitness:  0.6
Mean Generation 1 Fitness:  1.0
Mean Generation 2 Fitness:  1.0
Mean Generation 3 Fitness:  1.0
Mean Generation 4 Fitness:  1.0
Mean Generation 5 Fitness:  1.0
Mean Generation 6 Fitness:  1.0


In [24]:
print('Final Population Scores: ', evaluations)
print('-------------------------------------')
print()
print('Final Population Instructions: ') 
print()
print(prompt_instructions)
print('-------------------------------------')
print()
print('final Answers:')
model_ans

Final Population Scores:  [1. 1. 1. 1. 1.]
-------------------------------------

Final Population Instructions: 

['Consider using an algorithm or formula that is most efficient in solving this type of math problem. Provide the final answer as a number.'
 'Break down the math problem step-by-step and provide the final numeric solution.'
 'You receive a mysterious letter with a math word problem written on it. The problem seems simple enough: "If a train leaves Station A at '
 'People who are good at creative thinking would approach this mutation question by thinking outside the box and exploring different possib'
 'Solve the math word problem, giving your answer as an arabic numeral']
-------------------------------------

final Answers:


['To solve this problem, we can use the formula:\n\nTotal pieces of pie - Remaining pieces of pie = Pieces taken by guests\n\nIn this case, the total pieces of pie are 5 pies * 8 pieces per pie = 40 pieces.\n\nSo, the equation becomes:\n\n40 - 14 = Pieces taken by guests\n\nSimplifying the equation:\n\n26 = Pieces taken by guests\n\nTherefore, the guests took 26 pieces of pie.',
 'Stakeholders involved in this situation:\n1. Ted the T-Rex (the one planning to bring potato salad)\n2. Adult dinosaurs attending the picnic\n3. Child dinosaurs attending the picnic\n\nHow they would feel about this situation:\n1. Ted the T-Rex: Ted would want to ensure that he brings enough potato salad to feed everyone at the picnic. He would feel responsible for providing an adequate amount of food for both the adults and children.\n2. Adult dinosaurs: The adult dinosaurs would expect to have enough potato salad to satisfy their appetite. They would want to enjoy the picnic without worrying about running o

In [30]:
num_tokens_from_string(model_ans[2], 'cl100k_base')

285